In [ ]:
# per tweet

import re
import pandas as pd
import torch
import os

from torch import nn
from transformers import BertForSequenceClassification
from myDataSet import MyTokenizerDataset
from torch.utils.data import DataLoader
from newfit import train, test

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)



learning_rate = 1e-4
split_ratio=0.2
input_size = 768
epochs = 10
batch_size = 128
decay_rate = 0.95



trainPath = './data/traintweetswithlabel_1.csv'
train_csv=pd.read_csv(trainPath)
dfTrain=pd.DataFrame(train_csv)
print("loading train data")
train_data=MyTokenizerDataset(dfTrain)

# train_counts=[pd.value_counts(dfTrain['label'])[0],pd.value_counts(dfTrain['label'])[1]]
train_counts=pd.value_counts(dfTrain['label']).tolist()
print(train_counts)
train_weights= 1./ torch.tensor(train_counts, dtype=torch.float)
train_targets = train_data.getLabels()
train_samples_weights = train_weights[train_targets]
train_sampler = torch.utils.data.WeightedRandomSampler(weights=train_samples_weights, num_samples=len(train_samples_weights), replacement=True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, sampler=train_sampler, num_workers=4,pin_memory=True)


testPath = './data/testtweetswithlabel_1.csv'
test_csv=pd.read_csv(testPath)
dfTest=pd.DataFrame(test_csv)
print("loading test data")
test_data = MyTokenizerDataset(dfTest)

# test_counts=[pd.value_counts(dfTest['label'])[0],pd.value_counts(dfTest['label'])[1]]
test_counts=pd.value_counts(dfTest['label']).tolist()
print(test_counts)
test_weights= 1./ torch.tensor(test_counts, dtype=torch.float)
test_targets = test_data.getLabels()
test_samples_weights = test_weights[test_targets]
test_sampler = torch.utils.data.WeightedRandomSampler(weights=test_samples_weights, num_samples=len(test_samples_weights), replacement=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, sampler=test_sampler, num_workers=4,pin_memory=True)

lrs=[1e-4,5e-5,2e-5,1e-5,5e-6]
for lr in lrs:
    learning_rate=lr
    net = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 2, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    # Tell pytorch to run this model on the GPU.
    net.cuda()

    optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate,eps=1e-8)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)

    test(net,test_loader,device)
    train(net, train_loader, test_loader,optimizer, scheduler, device, epochs)
    test(net,test_loader,device)

In [ ]:
# per user

import re
import pandas as pd
import torch
import os

from torch import nn
from transformers import BertForSequenceClassification
from myDataSet import MyTokenizerDataset
from torch.utils.data import DataLoader
from fit import train, test

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

learning_rate = 1e-4
split_ratio=0.2
input_size = 768
epochs = 100
batch_size = 16
decay_rate = 0.95

trainPath = './data/mergetraintweetswithlabel_1.csv'
train_csv=pd.read_csv(trainPath)
dfTrain=pd.DataFrame(train_csv)
dfTrainauc=dfTrain.loc[dfTrain['label']==1]
dfTrain=pd.concat([dfTrain,dfTrainauc], ignore_index=True)
print("loading train data")
train_data=MyTokenizerDataset(dfTrain)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)


testPath = './data/mergetesttweetswithlabel_1.csv'
test_csv=pd.read_csv(testPath)
dfTest=pd.DataFrame(test_csv)
# dfTestauc=dfTest.loc[dfTest['label']==1]
# dfTest=pd.concat([dfTest,dfTestauc], ignore_index=True)
print("loading test data")
test_data = MyTokenizerDataset(dfTest)

test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4,pin_memory=True)

lrs=[1e-7,5e-6]
for lr in lrs:
    learning_rate=lr
    net = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
        num_labels = 2, # The number of output labels--2 for binary classification.
                        # You can increase this for multi-class tasks.   
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.
    )

    # Tell pytorch to run this model on the GPU.
    net.cuda()

    optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate,eps=1e-8)
    scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_rate)

    test(net,test_loader,device)
    train(net, train_loader, test_loader,optimizer, scheduler, device, epochs)
    test(net,test_loader,device)